In [153]:
"""

LINKS:

- https://github.com/sagemath/sage/issues/38456
- https://github.com/sagemath/sage/pull/38455

- https://mathoverflow.net/questions/271932/formula-for-the-frobenius-schur-indicator-of-a-finite-group
- https://math.stackexchange.com/questions/832173/structure-of-g-invariant-bilinear-forms-over-finite-fields

""";

In [152]:
"""
NOTES:

For a unitary change-of-basis for a rep'n \rho of S_n over finite fields, we need:

Q1. find a S_n-invariant symmetric bilinear form w/ associated matrix U
Q2. factor U = A^*A, where * denotes conjugate-transpose w.r.t. x |--> x^q conjugation

A1. let U be unknown in variables over a polynomial ring over GF(q^2). use linear algebra to solve \rho(g)^T U \overline{\rho(g)} == U
A2. translate the GAP `forms` package which uses a conjugate symmetric form of Gaussian elimination. this extends the Cholesky decomposition to non lower triangular matrices

Then \tilde{\rho}(g) = A\rho(g)A.inverse() is unitary for all g \in G.

n.b. this code is now merged into SageMath via PR #38455.

""";

In [110]:
def invariant_symmetric_bilinear_matrix(SGA,partition,symmetric=False):
    """
    Computes the matrix of a S_n-invariant symmetric bilinear form.
    Sets up and solves system of linear equations based on writing U as an unknown in polynomial ring generators. 
    The equations are \rho(g)^T*U*\overline{\rho(g)} = \lambda_g*U where \lambda_g = \det(\rho(g))\overline{\det(\rho(g))}.
    The variables for U can be extracted to yield a matrix over GF(q^2) for each g.
    These are stacked to get the overall system, and we find the one dim'l null space to get a solution vector, and format as a matrix.
    NOTE: one could also form the Kroenecker products \rho(g) \otimes \rho(g)^{-1 T} to explicitly obtain the system.
    NOTE: the solution space is typically 1 dim'l, except in "modular" cases where p divides |G|, and there is multiplicity in the decomposition factors.
    BUG: this does work for the 2-element generating set {(1 2), (1 2 ... n)} of the symmetric group, and it should
    """
    n = sum(partition)
    G = Permutations(n)
    G_gens = [G([2, 1]), G([i%n+1 for i in range(1,n+1)])] #small gen set {(1 2), (1 2 ... n)}. can use G.gens() generally
    F = SGA.base_ring()
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    d_rho = specht_module.dimension()
    if symmetric:
        R = PolynomialRing(F, 'u', d_rho * (d_rho + 1) // 2)
    else:
        R = PolynomialRing(F, 'u', d_rho**2)
    U_vars = R.gens()
    if symmetric:
        U_temp = matrix(R, d_rho, d_rho, lambda i, j: U_vars[i * d_rho - (i * (i + 1)) // 2 + j] if i <= j else 0)
        U_temp += U_temp.transpose() - diagonal_matrix(U_temp.diagonal())
    else:
        U_temp = matrix(R, d_rho, d_rho, U_vars)

    def augmented_matrix(g):
        rho_g = rho(g)
        equation_matrix = rho_g.transpose() * U_temp * rho_g.conjugate() - U_temp
        augmented_system = []
        for i in range(d_rho):
            for j in range(d_rho):
                linear_expression = equation_matrix[i, j]
                row = [linear_expression.coefficient(u) for u in U_vars]
                augmented_system.append(row)
        return augmented_system

    total_system = sum((augmented_matrix(g) for g in G_gens), [])
    null_space = matrix(F, total_system).right_kernel()
    U_list = []
    if symmetric:
        for B in null_space.basis():
            upper_triangle = matrix(F, d_rho, d_rho, lambda i, j: B[i * d_rho - (i * (i + 1)) // 2 + j] if i <= j else 0)
            U_list.append(upper_triangle + upper_triangle.transpose() - diagonal_matrix(upper_triangle.diagonal()))
    else:
        U_list = [matrix(F, d_rho, d_rho, B) for B in null_space.basis()]
    return U_list

In [85]:
#define a general bi/bilinear form with the matrix U
def bilinear_form(x,y,U):
    return x.transpose()*U*y

In [101]:
#ensure the resulting form is G-invariant, symmetric, bilinear by symbolic verification
def check_form_properties(q,partition,symmetric=False):
    #define the representation matrix corresponding to q, partition
    F = GF(q**2)
    SGA = SymmetricGroupAlgebra(F,sum(partition))
    SM = SGA.specht_module(partition)
    rho = SM.representation_matrix
    d_rho = SM.dimension()
    G = SGA.group()

    #define variables as polynomial generators
    R_xy = PolynomialRing(F, d_rho, var_array='x,y')
    x = matrix([R_xy.gens()[2*i] for i in range(d_rho)]).transpose()
    y = matrix([R_xy.gens()[2*i+1] for i in range(d_rho)]).transpose()
    R_xy_lambda = PolynomialRing(R_xy,'lambda')
    lambda_ = R_xy_lambda.gens()[0]

    #compute the bilinear form matrix. coerce over polynomial ring
    U_mats = invariant_symmetric_bilinear_matrix(SGA,partition,symmetric)
    if len(U_mats) > 1:
        print("Space of G-invariant symmetric bilinear forms has dimension > 1 for la=",partition)
        print("Dimension of space=",len(U_mats))
    U_mat = U_mats[0]
    U_form = matrix(R_xy_lambda,U_mat)
    
    #check symmetric property
    symmetric = bilinear_form(x,y,U_form) == bilinear_form(y,x,U_form)
    
    #check G-invariance property
    G_invariant = all(bilinear_form(rho(g)*x,rho(g)*y,U_form) == bilinear_form(x,y,U_form) for g in G)
    
    #check bilinear property. ISSUE: lambda_^q is a power of the ring generator, i.e. doesn't simplify.
    first_arg = bilinear_form(lambda_*x,y,U_form) == lambda_*bilinear_form(x,y,U_form)
    second_arg = bilinear_form(x,lambda_*y,U_form) == lambda_*bilinear_form(x,y,U_form) #need to amend for conjugation
    bilinear = first_arg and second_arg

    return symmetric and G_invariant and bilinear

In [102]:
#for u in GF(q), we can factor as u=aa^* using gen. z and modular arithmetic
def conj_square_root(u):
    if u == 0:
        return 0  # Special case for 0
    z = F.multiplicative_generator()
    k = u.log(z)  # Compute discrete log of u to the base z
    if k % (q+1) != 0:
        raise ValueError("Unable to factor: u is not in base field GF(q)")
    return z ** (k//(q+1))

In [103]:
def hermitian_decomposition(mat):
    """
    Diagonalizes a Hermitian matrix over a finite field.
    Returns the base change matrix and the rank of the Hermitian form.
    
    Arguments:
        mat: The Gram matrix of a Hermitian form (Sage matrix object)
        F: The finite field (GF(q))
    
    Returns:
        D: The base change matrix
        r: The number of non-zero rows in D*mat*D^T
    """

    F = mat.base_ring()
    n = mat.nrows()
    q = sqrt(F.order())

    if mat.nrows() == 1 and mat.ncols() == 1:
        return matrix(F,[conj_square_root(mat[0][0])])

    A = copy(mat)
    D = identity_matrix(F, n)
    row = 0

    # Diagonalize A
    while True:
        row += 1

        # Look for a non-zero element on the main diagonal, starting from `row`
        i = row - 1  # Adjust for zero-based indexing in Sage
        while i < n and A[i, i].is_zero():
            i += 1

        if i == row - 1:
            # Do nothing since A[row, row] != 0
            pass
        elif i < n:
            # Swap to ensure A[row, row] != 0
            A.swap_rows(row - 1, i)
            A.swap_columns(row - 1, i)
            D.swap_rows(row - 1, i)
        else:
            # All entries on the main diagonal are zero; look for an off-diagonal element
            i = row - 1
            while i < n - 1:
                k = i + 1
                while k < n and A[i, k].is_zero():
                    k += 1
                if k == n:
                    i += 1
                else:
                    break

            if i == n - 1:
                # All elements are zero; terminate
                row -= 1
                r = row
                break

            # Fetch the non-zero element and place it at A[row, row + 1]
            if i != row - 1:
                A.swap_rows(row - 1, i)
                A.swap_columns(row - 1, i)
                D.swap_rows(row - 1, i)

            A.swap_rows(row, k)
            A.swap_columns(row, k)
            D.swap_rows(row, k)

            b = A[row, row - 1]**(-1)
            A.add_multiple_of_column(row - 1, row, b**q)
            A.add_multiple_of_row(row - 1, row, b)
            D.add_multiple_of_row(row - 1, row, b)

        # Eliminate below-diagonal entries in the current column
        a = -A[row - 1, row - 1]**(-1)
        for i in range(row, n):
            b = A[i, row - 1] * a
            if not b.is_zero():
                A.add_multiple_of_column(i,row - 1, b**q)
                A.add_multiple_of_row(i, row - 1, b)
                D.add_multiple_of_row(i, row - 1, b)

        if row == n - 1:
            break

    # Count how many variables have been used
    if row == n - 1:
        if not A[n - 1, n - 1].is_zero():
            r = n
        else:
            r = n - 1

    # Normalize diagonal elements to 1
    for i in range(r):
        a = A[i, i]
        if not a.is_one():
            # Find an element `b` such that `b*b^t = b^(t+1) = a`
            b = conj_square_root(a)
            D.rescale_row(i, 1 / b)

    return D.inverse()

In [104]:
#define the Fourier coefficient at the rep'n specht_module corresponding to partition
def hat(g,partition,unitary=False,SGA=None):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        U = invariant_symmetric_bilinear_matrix(SGA,partition)[0]
        A = hermitian_decomposition(U)
        sqrt_unitary_factor = sqrt(F(specht_module.dimension()/G.cardinality()))
        return sqrt_unitary_factor*A.H*rho(g)*A.H.inverse()
    else:
        return rho(g)

In [105]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
def dft(unitary=False):
    fourier_transform = [[x for partition in Partitions(G.degree()) for x in hat(g, partition, unitary, SGA).list()] for g in G]
    if unitary:
        dft_matrix = matrix(F,fourier_transform).transpose()
        sign_diag = (dft_matrix*dft_matrix.H).diagonal()
        factor_diag_inv = diagonal_matrix([~conj_square_root(d) for d in sign_diag])
        return factor_diag_inv*dft_matrix
    else:
        return matrix(fourier_transform).transpose()

In [4]:
#define the representation matrix corresponding to q, partition
#define generators of multiplicative groups of GF(q) and GF(q^2), w and z respectively
q = 7; la = [3,1,1]; n=sum(la)
F = GF(q**2)
SGA = SymmetricGroupAlgebra(F,n)
SM = SGA.specht_module(la)
rho = SM.representation_matrix
d_rho = SM.dimension()
G = SGA.group()
z = F.gen()

In [5]:
#BUG: setting the base ring of the representation matrix to GF(q^2) doesn't work, results in rational field instead
rho = SymmetricGroupRepresentation(la,ring=GF(7**2))._representation_matrix_uncached; rho(Permutation([2,1,3,4,5])).base_ring()

Rational Field

In [43]:
#to explain q=2, la=[3,1,1] note that the one of the composition factors (the irreducible quotients from any composition series)
#has multiplicity 2. we see this by looking at the Brauer character of the composition factor, and see (1,1,1) appears twice.
SGA_5_GF4 = SymmetricGroup(5).algebra(GF(2**2))
for la in Partitions(5):
    M = SGA_5_GF4.specht_module(la)
    print(la, [V.brauer_character() for V in M.composition_factors()])

[5] [(1, 1, 1)]
[4, 1] [(4, 1, -1)]
[3, 2] [(4, -2, -1), (1, 1, 1)]
[3, 1, 1] [(1, 1, 1), (4, -2, -1), (1, 1, 1)]
[2, 2, 1] [(1, 1, 1), (4, -2, -1)]
[2, 1, 1, 1] [(4, 1, -1)]
[1, 1, 1, 1, 1] [(1, 1, 1)]


In [112]:
#NOTE: for q=2, la=[3,1,1] we get a two dim'l space
invariant_symmetric_bilinear_matrix(SGA_5_GF4,[3,1,1],symmetric=True)

[
[1 1 1 1 1 0]  [0 0 0 0 0 1]
[1 1 1 1 0 1]  [0 0 0 0 1 0]
[1 1 1 0 1 1]  [0 0 0 1 0 0]
[1 1 0 1 1 1]  [0 0 1 0 0 0]
[1 0 1 1 1 1]  [0 1 0 0 0 0]
[0 1 1 1 1 1], [1 0 0 0 0 0]
]

In [96]:
#print the matrix associated to the bilinear form for each partition
#note q needs to be large enough. p \nmid n! seems sufficient
for partition in Partitions(n):
    U_mats = invariant_symmetric_bilinear_matrix(SGA,partition,symmetric=False)
    if len(U_mats) > 1:
        print("space of G-invariant symmetric bilinear forms has dimension greater than 1")
    print("la =",partition)
    print("q =",q)
    print(U_mats[0])
    print("--------")

la = [5]
q = 7
[1]
--------
la = [4, 1]
q = 7
[1 4 4 4]
[4 1 4 4]
[4 4 1 4]
[4 4 4 1]
--------
la = [3, 2]
q = 7
[1 4 4 2 5]
[4 1 2 4 2]
[4 2 1 4 2]
[2 4 4 1 4]
[5 2 2 4 1]
--------
la = [3, 1, 1]
q = 7
[1 5 2 5 2 0]
[5 1 5 5 0 2]
[2 5 1 0 5 2]
[5 5 0 1 5 5]
[2 0 5 5 1 5]
[0 2 2 5 5 1]
--------
la = [2, 2, 1]
q = 7
[1 5 2 2 5]
[5 1 5 5 5]
[2 5 1 5 5]
[2 5 5 1 5]
[5 5 5 5 1]
--------
la = [2, 1, 1, 1]
q = 7
[1 2 5 2]
[2 1 2 5]
[5 2 1 2]
[2 5 2 1]
--------
la = [1, 1, 1, 1, 1]
q = 7
[1]
--------


In [108]:
#print the checks for pairs (p,la) where p is a prime and la is a partition of n
for p in Primes()[:3]:
    print(f"p={p}")
    for n in range(3,6):
        print([(la,check_form_properties(p,la,True)) for la in Partitions(n)])
    print("---------")

p=2
[([3], True), ([2, 1], True), ([1, 1, 1], True)]
[([4], True), ([3, 1], True), ([2, 2], True), ([2, 1, 1], False), ([1, 1, 1, 1], True)]
Space of G-invariant symmetric bilinear forms has dimension > 1 for la= [3, 1, 1]
Dimension of space= 2
[([5], True), ([4, 1], False), ([3, 2], False), ([3, 1, 1], False), ([2, 2, 1], False), ([2, 1, 1, 1], False), ([1, 1, 1, 1, 1], True)]
---------
p=3
[([3], True), ([2, 1], False), ([1, 1, 1], True)]
[([4], True), ([3, 1], False), ([2, 2], False), ([2, 1, 1], True), ([1, 1, 1, 1], True)]
[([5], True), ([4, 1], False), ([3, 2], False), ([3, 1, 1], False), ([2, 2, 1], False), ([2, 1, 1, 1], False), ([1, 1, 1, 1, 1], True)]
---------
p=5
[([3], True), ([2, 1], False), ([1, 1, 1], True)]
[([4], True), ([3, 1], False), ([2, 2], False), ([2, 1, 1], False), ([1, 1, 1, 1], True)]
[([5], True), ([4, 1], False), ([3, 2], False), ([3, 1, 1], False), ([2, 2, 1], False), ([2, 1, 1, 1], False), ([1, 1, 1, 1, 1], True)]
---------


In [144]:
#compute the S_n-invariant symmetric bilinear form associated to (q,la)
U = invariant_symmetric_bilinear_matrix(SGA,la)[0]; U

[1 5 2 5 2 0]
[5 1 5 5 0 2]
[2 5 1 0 5 2]
[5 5 0 1 5 5]
[2 0 5 5 1 5]
[0 2 2 5 5 1]

In [145]:
#compute the factorization U = AA^*
A = hermitian_decomposition(U); print(A)
U == A*A.H

[       1        0        0        0        0        0]
[       5 2*z2 + 6        0        0        0        0]
[       2   z2 + 3       z2        0        0        0]
[       5 4*z2 + 5        0   z2 + 4        0        0]
[       2 2*z2 + 6     2*z2 4*z2 + 2   z2 + 1        0]
[       0   z2 + 3     5*z2 4*z2 + 2 5*z2 + 5 5*z2 + 4]


True

In [146]:
#verify explicitly the representation is unitary
g = G[1]
rho_unitary = A.H*matrix(F,rho(g))*A.H.inverse(); print(rho_unitary)
rho_unitary*rho_unitary.H == 1

[       1        0        0        0        0        0]
[       0        2        0 3*z2 + 6        0        0]
[       0        0        2        0 2*z2 + 6        0]
[       0 4*z2 + 2        0        5        0        0]
[       0        0 5*z2 + 1        0        5        0]
[       0        0        0        0        0        6]


True

In [70]:
#construct the unitary representations as \tilde{\rho}(g) = A^*\rho(g)A^*.inverse()
all_unitary = True
for g in G:
    tilde_rho_g = A.H*rho(g)*A.H.inverse()
    unitary_check = tilde_rho_g.H*tilde_rho_g == 1
    all_unitary = all_unitary and unitary_check
print(all_unitary)

True


In [43]:
#compute the unitary DFT
unitary_dft = dft(unitary=True); print(list(unitary_dft))

[(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), (2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 3*z2 + 1, 3*z2 + 1, z2 + 5, z2 + 5, z2 + 5, z2 + 5, 3*z2 + 1, 3*z2 + 1, z2 + 5, z2 + 5, z2 + 5, z2 + 5, 3*z2 + 1, 3*z2 + 1, z2 + 5, z2 + 5, z2 + 5, z2 + 5, 0, 0, 0, 0, 0, 0, 4*z2 + 6, 4*z2 + 6, 6*z2 + 2, 6*z2 + 2, 6*z2 + 2, 6*z2 + 2, 4*z2 + 6, 4*z2 + 6, 6*

In [44]:
#verify the resulting DFT is unitary
unitary_dft*unitary_dft.H == identity_matrix(SGA.group().cardinality())

True

In [128]:
def naive_invariant_symmetric_bilinear_matrix(q,partition):
    r"""
    alternatively, we can just average over the usual dot product when p does not divide |G|
    it is slower than the other method, and does not work when p divides |G|
    in fact, we get a scalar multiple of the other solution since the space is one dimensional
    """
    n = sum(partition)
    G = SymmetricGroup(n)
    F = GF(q)
    V = G.algebra(F).specht_module(partition)
    d_rho = V.dimension()
    rho = V.representation_matrix
    
    # Define a symmetric bilinear form using the action
    def invariant_form(x, y):
        return sum((rho(g) * vector(x)).dot_product(rho(g) * vector(y)) for g in G) / G.order()
    
    basis = V.basis()
    n = len(basis)
    return matrix(F, n, n, lambda i, j: invariant_form(basis.list()[i], basis.list()[j]))

In [130]:
naive_invariant_symmetric_bilinear_matrix(5**2,[2,1,1])

[4 3 2]
[3 4 3]
[2 3 4]

In [114]:
SGA_GF49_S4 = SymmetricGroupAlgebra(GF(11**2),4)
invariant_symmetric_bilinear_matrix(SGA_GF49_S4,[2,1,1])

[
[1 4 7]
[4 1 4]
[7 4 1]
]

In [74]:
#define variables as polynomial generators
R_xy = PolynomialRing(F, d_rho, var_array='x,y')
x = matrix([R_xy.gens()[2*i] for i in range(d_rho)]).transpose()
y = matrix([R_xy.gens()[2*i+1] for i in range(d_rho)]).transpose()
R_xy_lambda = PolynomialRing(R_xy,'lambda')
lambda_ = R_xy_lambda.gens()[0]

U_form = matrix(R_xy_lambda,M)

#check symmetric property
symmetric = bilinear_form(x,y,U_form) == bilinear_form(y,x,U_form)

#check G-invariance property
G_invariant = all(bilinear_form(rho(g)*x,rho(g)*y,U_form) == bilinear_form(x,y,U_form) for g in G)

#check bilinear property. ISSUE: lambda_^q is a power of the ring generator, i.e. doesn't simplify.
first_arg = bilinear_form(lambda_*x,y,U_form) == lambda_*bilinear_form(x,y,U_form)
second_arg = bilinear_form(x,lambda_*y,U_form) == lambda_*bilinear_form(x,y,U_form) #need to amend for conjugation
bilinear = first_arg and second_arg

symmetric and G_invariant and bilinear

True